Group Name: AG 19

Student Name (Student ID):

1. Lee Han Jian (A0248322J)

2. LI SHENGDONG (xxxxxxx)

3. ZHENG WENGRUI (xxxxxxx)

# Question 1

Consider the maze shown below. The Maze has 16 rows and 24 columns The objective is to find a shortest path from cell $S$ to cell $G$.


![Maze](https://github.com/Hanjianlee/IT5005/blob/main/assignment1/Maze_Assignment_1.jpg?raw=1)


The agent can take four actions in each cell: 'RIGHT', 'DOWN', 'UP', 'LEFT'.  

Each cell is represented as $(x,y)$, where $x$ indicates row number and $y$ indicates column number. Action 'UP' takes the agent from cell $(x,y)$ to $(x+1,y)$. Action 'DOWN' takes the agent from cell $(x,y)$ to $(x-1,y)$. Action 'RIGHT' takes the agent from cell $(x,y)$ to $(x,y+1)$. Action 'LEFT' takes the agent from cell $(x,y)$ to $(x,y-1)$. The triplet $(s,a,s')$  indicates that taking action $a$ at state $s$ leads to state $s'$. Actions 'LEFT' or 'RIGHT' cost 10 units for all $(s,a,s')$. Actions 'UP' or 'DOWN' cost 1 unit for all  $(s,a,s')$.  The agent cannot move into cells that are shaded. Assume that the agent knows the boundaries of the maze and has full observability. Consequently, at the bottom (row 0) and top (row 15), the agent will not take actions 'DOWN' and 'UP', respectively; at left (column 0) and right (column 23) columns, the agent will not take 'LEFT' and 'RIGHT' actions, respectively. Similalry, the agent will not take actions that lead to shaded region in the maze.

## **Q1.a: Class Maze(Problem)** [3 Marks]

Write a Maze class to create a model for this problem. You should not use an explicit state space model. The modelling should inherit the abstract class 'Problem' (given below). With the problem formulation, find the shortest path from S to G cell. Propose and implement multiple heuristics (at least two heuristics) for informed search algorithms. 

## **Q1.b: Analysis of the Algorithms** [7 Marks]

1. Solve the above Maze problem using the following algorithms

    a. Breadth-First Search

    b. Depth-First Search with Cycle-Check

    c. Iterative-Deepening Search with Cycle-Check

    d. Uniform-Cost Search

    e. A* Search 

    f. Greedy Best-first Search

    g. Any other variants for search algorithms that are not discussed in the class (bonus/optional question) 

2. Identify the number of nodes generated, number of nodes expanded, maximum frontier size, and path-cost for the above algorithms. 
 
3. Compare the performance of informed search algorithms with proposed heuristics. Identify the best performing heuristic and explain.
 
4. Draw a bar plot comparing the statistics of the algorithms and explain the results. 

Note 1: You must follow the problem formulation discussed in the class. A abstract class for Problem and Node definition is presented below. The search tree generation should follow the template discussed in the class (i.e., Node class, expand methods, etc.). 

Note 2: If you are borrowing a block of code (for example, helper functions or data structures, etc.) from AIMA4e repository, you have to acknowledge it in the code. 

Note 3: The code should be written in a single jupyter notebook file.

In [2]:
class Problem:
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

In [3]:
# Use the following Node class to generate search tree
import math
class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost 
    def get_details(self):
        print(self.state,self.parent,self.action,self.path_cost)


In [14]:
from collections import deque
class Maze(Problem):
    #Your code goes here (16x24 Board)
    def __init__(self):
        self.initial = (8,10) # Start
        self.goal = (11,9) # Goal 
        self.hueristic = 1
        self.up_cost = self.down_cost = 1
        self.left_cost = self.right_cost = 10
        # Shaded regions
        self.shaded_regions = set()
        self.add_shadded_region(14,9)

        #Column 1 @ column 10
        self.add_shadded_region(13,10)
        self.add_shadded_region(12,10)
        self.add_shadded_region(11,10)
        self.add_shadded_region(10,10)

        #Column 2 @ column
        self.add_shadded_region(10,9)
        self.add_shadded_region(9,9)
        self.add_shadded_region(8,9)
        self.add_shadded_region(7,9)
        self.add_shadded_region(6,9)
 
        # SquareBox
        self.add_shadded_region(11,12)
        self.add_shadded_region(10,12)
        self.add_shadded_region(11,13)
        self.add_shadded_region(10,13)

        self.check = Node(self.initial,None,None,1)
        # Print start Node State
        if self.check.state not in self.shaded_regions:
            print("Good start! :)")
        else: 
            print("Bad Start! :(")
    
    def add_shadded_region(self,r: int,c: int):
        if not isinstance(r,int) or not isinstance(c,int):
            raise AttributeError
        return self.shaded_regions.add((r,c))
    
    def actions(self):
        return ['UP','DOWN','LEFT','RIGHT']

    def result(self, state:tuple, action: str): 
        if  action == 'UP':
            if state[0] == 15:
                return None 
            new_state = (state[0]+1,state[1])
            if new_state in self.shaded_regions:
                return None
        elif action == 'DOWN':
            if state[0] == 0 :
                return None     
            new_state = (state[0]-1,state[1])
            if new_state in self.shaded_regions:
                return None
        elif action == 'LEFT':
            if state[1] == 0:
                return None 
            new_state = (state[0],state[1]-1)
            if new_state in self.shaded_regions:
                return None
        elif action == 'RIGHT':
            if state[1] == 23:
                return None 
            new_state =(state[0],state[1]+1)
            if new_state in self.shaded_regions:
                return None
        return Node(new_state,action,self.up_cost)

    def expand(self,state):
        nodes = []
        for action in self.actions():
            result = self.result(state,action)
            if result is not None:
                nodes.append(result)
        return nodes

class BFS(Maze):
  def search(self):
      print("[1. BFS]")
      node = Node(self.initial)
      if self.is_goal(node.state):
          return node
      frontier = deque([node]) # FIFO
      reached = {self.initial}
      count = 0
      while len(frontier):
          count += 1
          node = frontier.popleft()
          # TODO: loop through expanded nodes ~~ 
          for child_node in  self.expand(node.state):
              if self.is_goal(child_node.state): 
                  print(count)
                  return child_node
              if child_node.state not in reached:
                  reached.add(child_node.state)
                  frontier.append(child_node)
      return 'Error'
        
class DFS(Maze):
  def search(self):
      print("[2. DFS]")
      node = Node(self.initial)
      if self.is_goal(node.state):
          return node
      frontier = [node] # LIFO
      reached = {self.initial}
      count = 0
      while len(frontier):
          count += 1
          node = frontier.pop()
          # TODO: loop through expanded nodes ~~ 
          for child_node in  self.expand(node.state):
              if self.is_goal(child_node.state): 
                  print(count)
                  return child_node
              if child_node.state not in reached:
                  reached.add(child_node.state)
                  frontier.append(child_node)
      print(count)
      return 'Error'

bfs = BFS()
bfs.search()
dfs = DFS()
dfs.search()


Good start! :)
[1. BFS]
166
Good start! :)
[2. DFS]
180


<(11, 9)>